Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10,6)

In [ ]:
df = pd.read_csv('/content/wcmatches.csv')
df.head()

In [ ]:
df.info()
df.describe()
df.columns

In [ ]:
df['stage'].unique()

In [ ]:
finals = df[df['stage'] == 'Final']
finals[['year', 'home_team', 'away_team', 'home_score', 'away_score', 'winning_team']]

In [ ]:
champions = finals[['year', 'winning_team']].rename(columns={'winning_team': 'champion'})
champions.head()

In [ ]:
stats_list = []

for index, row in champions.iterrows():
    year = row['year']
    team = row['champion']

    matches = df[
        (df['year'] == year) &
        ((df['home_team'] == team) | (df['away_team'] == team))
    ]

    gols_marcados = (
        matches[matches['home_team'] == team]['home_score'].sum() +
        matches[matches['away_team'] == team]['away_score'].sum()
    )

    gols_sofridos = (
        matches[matches['home_team'] == team]['away_score'].sum() +
        matches[matches['away_team'] == team]['home_score'].sum()
    )

    jogos = len(matches)

    stats_list.append({
        'year': year,
        'champion': team,
        'matches': jogos,
        'goals_scored': gols_marcados,
        'goals_conceded': gols_sofridos,
        'goal_difference': gols_marcados - gols_sofridos,
        'goals_per_game': gols_marcados / jogos,
        'conceded_per_game': gols_sofridos / jogos
    })

champions_stats = pd.DataFrame(stats_list)
champions_stats.head()

Campeões com melhor ataque

In [ ]:
champions_stats.sort_values('goals_per_game', ascending=False).head(10)

Campeões com melhor defesa

In [ ]:
champions_stats.sort_values('conceded_per_game').head(10)

Evolução da média de gols ao longo do tempo

In [ ]:
plt.plot(champions_stats['year'], champions_stats['goals_per_game'])
plt.title('Evolução da Média de Gols dos Campeões')
plt.xlabel('Ano')
plt.ylabel('Gols por Jogo')
plt.show()

Heatmap de Correlação

In [ ]:
sns.heatmap(champions_stats.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Correlação Entre Métricas dos Campeões')
plt.show()

In [ ]:
champions_sorted = champions_stats.sort_values('goals_per_game', ascending=True)

plt.figure(figsize=(10,8))
plt.barh(champions_sorted['champion'] + " (" + champions_sorted['year'].astype(str) + ")",
         champions_sorted['goals_per_game'])

plt.title('Ranking de Campeões por Média de Gols por Jogo')
plt.xlabel('Gols por Jogo')
plt.ylabel('Seleção (Ano)')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(champions_stats['year'], champions_stats['goals_per_game'], marker='o')
plt.plot(champions_stats['year'], champions_stats['conceded_per_game'], marker='s')

plt.title('Evolução Ofensiva vs Defensiva dos Campeões')
plt.xlabel('Ano')
plt.ylabel('Gols por Jogo')
plt.legend(['Gols Marcados', 'Gols Sofridos'])
plt.show()

In [ ]:
plt.figure(figsize=(8,6))

plt.scatter(champions_stats['goals_per_game'],
            champions_stats['conceded_per_game'])

for i in range(len(champions_stats)):
    plt.text(champions_stats['goals_per_game'][i],
             champions_stats['conceded_per_game'][i],
             str(champions_stats['year'][i]))

plt.title('Ofensividade vs Solidez Defensiva dos Campeões')
plt.xlabel('Gols por Jogo')
plt.ylabel('Gols Sofridos por Jogo')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
corr = champions_stats.corr(numeric_only=True)

sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    linewidths=0.5,
    cmap="vlag"
)

plt.title("Correlation Matrix - World Cup Champions")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))

sns.scatterplot(
    data=champions_stats,
    x="goals_per_game",
    y="conceded_per_game",
    size="goal_difference",
    hue="year",
    palette="viridis",
    legend=False
)

plt.title("Offensive vs Defensive Performance of Champions")
plt.xlabel("Goals per Game")
plt.ylabel("Goals Conceded per Game")
plt.show()

In [ ]:
sns.lmplot(
    data=champions_stats,
    x="year",
    y="goals_per_game",
    aspect=1.5,
    height=6,
    scatter_kws={"s": 60}
)

plt.title("Trend of Goals per Game Over Time")
plt.show()